In [ ]:
from IPython.core.display import display, HTML

HTML(
    '''
    <script>
        code_show = false; 
        
        function code_toggle() {
            if (code_show){
                $('div.input').hide();
            } else {
                $('div.input').show();
            }
                code_show = !code_show
            }
            
        $( document ).ready(code_toggle);
    </script>
    
    <form action="javascript:code_toggle()">
        <input type="submit" value="Show code">
    </form>
    '''
)

<a id='index'></a>
# Single Site Simple Model Example

* [Input data](#input_data)
    * [GA parameters](#ga_params)
    * [Start date](#start_date)
    * [Product data](#product_data)
    * [Changeovers](#changeovers)
    * [Demand](#demand)
    * [Inventory target](#inventory_target)
* [Single objective problem 1](#so_problem)
    * [Objectives](#objectives)
    * [Inventory levels](#inventory)
    * [Campaigns gantt](#campaigns_gantt)
    * [Campaigns table](#campaigns_table)
    * [Batches table](#batches_table)
    * [Tasks gantt](#tasks_gantt)
    * [Tasks table](#tasks_table)
* [Single objective problem 2](#so_problem_2)
    * [Objectives](#objectives_2)
    * [Inventory levels](#inventory_2)
    * [Campaigns gantt](#campaigns_gantt_2)
    * [Campaigns table](#campaigns_table_2)
    * [Batches table](#batches_table_2)
    * [Tasks gantt](#tasks_gantt_2)
    * [Tasks table](#tasks_table_2)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import plotly.offline as opy
import plotly.graph_objs as go
opy.init_notebook_mode()

import utils
from biopharma_scheduling.deterministic.single_site import SingleSiteSimple

In [ ]:
if not os.path.exists('figures'):
    os.mkdir('figures')
    
colors = {
    'A': 'rgb(146, 208, 80)', 
    'B': 'rgb(179, 129, 217)', 
    'C': 'rgb(196, 189, 151)', 
    'D': 'rgb(255, 0, 0)'
}

def interactive_chart(df, colors: dict=None, title=''):
    data = [
        go.Scatter(
            x=df.index,
            y=df[product],
            name=product,
            marker=dict(
                color=colors[product]
            ) if colors else None,
            mode='lines+markers'
        ) for product in df
    ]

    layout = go.Layout(
        title=title,
        xaxis={
            'tickangle': -30,
        },
        yaxis={'title': 'Kg'}
    )

    return opy.iplot(go.Figure(data=go.Data(data), layout=layout))

<a id='input_data'></a>
## Input data

[back to top](#index)

In [ ]:
# How the input data is stored doesn't matter as long as Pandas DataFrames are used in the end
start_date = '2016-12-01'
kg_demand = pd.read_csv('data/kg_demand.csv', index_col='date')
kg_inventory_target = pd.read_csv('data/kg_inventory_target.csv', index_col='date')
product_data = pd.read_csv('data/product_data.csv')
changeover_days = pd.read_csv('data/changeover_days.csv')

display(HTML('<a id="ga_params"></a>'))
display(HTML('<h3>GA parameters</h3>'))
ga_params = pd.DataFrame(np.transpose([
    [20], 
    [1000], 
    [1000], 
    [1], 
    [0.820769],
    [0.141214], 
    [0.121224], 
    [0.213939], 
    [0.766782]
]), columns=[
    'num_runs',
    'popsize',
    'num_gens', 
    'starting_length',
    'p_xo', 
    'p_product_mut',
    'p_plus_batch_mut',
    'p_minus_batch_mut',
    'p_gene_swap'
], index=['value'])
display(ga_params.transpose())
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="start_date"></a>'))
display(HTML('<h3>Schedule start date</h3>'))
display(HTML('<h4>%s</h4>' % start_date))
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="product_data"></a>'))
display(HTML('<h3>Product data</h3>'))
product_data.kg_storage_limits = [250, 250, 250, 250]
product_data.shelf_life_days = [730, 730, 730, 730]
product_data.kg_opening_stock = [18.6, 0, 19.6, 32.0]
product_data.min_batches_per_campaign = [2, 2, 2, 3]
product_data.max_batches_per_campaign = [50, 50, 50, 30]
product_data.batches_multiples_of_per_campaign = [1, 1, 1, 3]
display(product_data.transpose())
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="changeovers"></a>'))
display(HTML('<h3>Product-dependent changeover data [days]</h3>'))
display(changeover_days)
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="demand"></a>'))
display(HTML('<h3>Product demand [kg]</h3>'))
display(kg_demand.transpose())
interactive_chart(kg_demand, colors)
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="inventory_target"></a>'))
display(HTML('<h3>Product inventory target [kg]</h3>'))
display(kg_inventory_target.transpose())
interactive_chart(kg_inventory_target, colors)
display(HTML('<a href=#index>back to top</a>'))

<a id='so_problem'></a>
## Single objective problem 1

* Objective:
    * Maximise ***total kg throughput***
<br><br>
* Constraints:
    * Keep ***total kg backlog*** at 0 kg
    * Keep ***total kg waste*** at 0 kg

[back to top](#index)

In [ ]:
objectives = {
    'total_kg_throughput': 1 # max
}

constraints = {
    'total_kg_backlog': [-1, 0], # <= 0
    'total_kg_waste': [-1, 0] # <= 0
}

model = SingleSiteSimple(
    num_runs=ga_params.num_runs[0],
    num_gens=ga_params.num_gens[0],
    popsize=ga_params.popsize[0],
    starting_length=ga_params.starting_length[0],
    p_xo=ga_params.p_xo[0],
    p_product_mut=ga_params.p_product_mut[0],
    p_plus_batch_mut=ga_params.p_plus_batch_mut[0],
    p_minus_batch_mut=ga_params.p_minus_batch_mut[0],
    p_gene_swap=ga_params.p_gene_swap[0],
    random_state=7, 
    num_threads=-1, 
    verbose=True
).fit(
    start_date,
    objectives,
    kg_demand,
    product_data,
    changeover_days,
    kg_inventory_target,
    constraints
)

In [ ]:
display(HTML('<a id="objectives"></a>'))
display(HTML('<h4>Objectives</h4>'))
display(model.schedules[0].objectives.transpose())
display(HTML('<a href=#index>back to top</a>'))

# display(HTML('<a id="inventory"></a>'))
# display(HTML('<h4>Inventory levels</h4>'))
# display(HTML('<p>Kg inventory after demand and waste have been accounted for.</p>'))
# utils.product_profiles(0, model)
# display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="campaigns_gantt"></a>'))
display(HTML('<h4>Campaigns gantt</h4>'))
display(model.schedules[0].campaigns_gantt(colors))
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="campaigns_table"></a>'))
display(HTML('<h4>Campaigns table</h4>'))
display(model.schedules[0].campaigns)
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="batches_table"></a>'))
display(HTML('<h4>Bathces table</h4>'))
display(model.schedules[0].batches)       
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="tasks_gantt"></a>'))
display(HTML('<h4>Tasks gantt</h4>'))
display(model.schedules[0].tasks_gantt(colors))   
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="tasks_table"></a>'))
display(HTML('<h4>Tasks table</h4>'))
display(model.schedules[0].tasks)    
display(HTML('<a href=#index>back to top</a>'))

<a id='so_problem_2'></a>
## Single objective problem 2

* Objective:
    * Minimise ***total kg inventory deficit***
<br><br>
* Constraints:
    * Keep ***total kg backlog*** at 0 kg
    * Keep ***total kg waste*** at 0 kg

[back to top](#index)

In [ ]:
objectives = {
    'total_kg_inventory_deficit': -1, # min
}

constraints = {
    'total_kg_backlog': [-1, 0], # <= 0
    'total_kg_waste': [-1, 0] # <= 0
}

model2 = SingleSiteSimple(
    num_runs=ga_params.num_runs[0],
    num_gens=ga_params.num_gens[0],
    popsize=ga_params.popsize[0],
    starting_length=ga_params.starting_length[0],
    p_xo=ga_params.p_xo[0],
    p_product_mut=ga_params.p_product_mut[0],
    p_plus_batch_mut=ga_params.p_plus_batch_mut[0],
    p_minus_batch_mut=ga_params.p_minus_batch_mut[0],
    p_gene_swap=ga_params.p_gene_swap[0],
    random_state=7, 
    num_threads=-1, 
    verbose=True
).fit(
    start_date,
    objectives,
    kg_demand,
    product_data,
    changeover_days,
    kg_inventory_target,
    constraints
)

In [ ]:
display(HTML('<a id="objectives_2"></a>'))
display(HTML('<h4>Objectives</h4>'))
display(model2.schedules[0].objectives.transpose())
display(HTML('<a href=#index>back to top</a>'))

# display(HTML('<a id="inventory_2"></a>'))
# display(HTML('<h4>Inventory levels</h4>'))
# display(HTML('<p>Kg inventory after demand and waste have been accounted for.</p>'))
# utils.product_profiles(0, model2)
# display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="campaigns_gantt_2"></a>'))
display(HTML('<h4>Campaigns gantt</h4>'))
display(model2.schedules[0].campaigns_gantt(colors))
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="campaigns_table_2"></a>'))
display(HTML('<h4>Campaigns table</h4>'))
display(model2.schedules[0].campaigns)
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="batches_table_2"></a>'))
display(HTML('<h4>Bathces table</h4>'))
display(model2.schedules[0].batches)       
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="tasks_gantt_2"></a>'))
display(HTML('<h4>Tasks gantt</h4>'))
display(model2.schedules[0].tasks_gantt(colors))   
display(HTML('<a href=#index>back to top</a>'))

display(HTML('<a id="tasks_table_2"></a>'))
display(HTML('<h4>Tasks table</h4>'))
display(model2.schedules[0].tasks)    
display(HTML('<a href=#index>back to top</a>'))